In [4]:
import pandas as pd
from sqlalchemy import create_engine
import mysql.connector
pd.options.mode.use_inf_as_na = True


### Store CSV into DataFrame

In [5]:
#define first source of data
csv_file = "Resources/GDP_Huge_Master.csv"
#convert to df
metroGDPdata_df = pd.read_csv(csv_file, dtype={'': int, 'Region': str, 'IndustryId': int })
#replace non-numeric values with 0
metroGDPdata_df = metroGDPdata_df.replace(to_replace={'(D)': 0, '(L)': 0, '(NaN)':0})
metroGDPdata_df

FileNotFoundError: File b'Resources/GDP_Huge_Master.csv' does not exist

### Create new data with select columns

In [78]:
edited_metroGDPdata_df = metroGDPdata_df[['GeoName','IndustryId','Description','2005','2006','2007','2008','2009','2010']].copy()
edited_metroGDPdata_df["IndustryId"] = edited_metroGDPdata_df["IndustryId"].apply(pd.to_numeric, errors='coerce')
edited_metroGDPdata_df

,GeoName,IndustryId,Description,2005,2006,2007,2008,2009,2010
0,United States (Metropolitan Portion),1.0,All industry total,11790834,12485329,13054952,13229897,12956728,13418482
1,United States (Metropolitan Portion),2.0,Private industries,10339935,10963791,11450487,11552249,11229026,11633016
2,United States (Metropolitan Portion),3.0,"Agriculture, forestry, fishing, and hunting",67155,69507,74544,75104,70163,80025
3,United States (Metropolitan Portion),4.0,Farms,52251,51217,56385,57076,52252,60446
4,United States (Metropolitan Portion),5.0,"Forestry, fishing, and related activities",14904,18289,18160,18027,17911,19579
5,United States (Metropolitan Portion),6.0,"Mining, quarrying, and oil and gas extraction",174850,207584,237231,304781,210566,236802
6,United States (Metropolitan Portion),7.0,Oil and gas extraction,134273,152595,174506,236921,154274,173088
7,United States (Metropolitan Portion),8.0,Mining (except oil and gas),21632,25417,28077,28450,29081,34092
8,United States (Metropolitan Portion),9.0,Support activities for mining,18945,29572,34648,39411,27211,29623
9,United States (Metropolitan Portion),10.0,Utilities,170177,194985,198786,200584,210494,223712


In [87]:
metroGDPdatabysector_df = edited_metroGDPdata_df.loc[edited_metroGDPdata_df['IndustryId'].isin(['82','89','90'])]
# metroGDPdatabysector_df = metroGDPdatabysector_df.groupby('GeoName').filter(lambda x : len(x)==8)
# metroGDPdatabysector_df.groupby('GeoName')


In [88]:

metroGDPdatabysector_df['Dollar_Growth'] = metroGDPdatabysector_df['2010'].astype('float64') - metroGDPdatabysector_df['2005'].astype('float64')
metroGDPdatabysector_df['Percentage_Growth'] = (metroGDPdatabysector_df['Growth 05 to 10'].astype('float64') // metroGDPdatabysector_df['2005'].astype('float64')) * 100
metroGDPdatabysector_df1 = metroGDPdatabysector_df
metroGDPdatabysector_df1.reset_index(drop=False,inplace=True)
metroGDPdatabysector_df1.reset_index(drop=False,inplace=True)


C:\Users\micha\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


KeyError: 'Growth 05 to 10'

In [84]:
metroGDPdatabysector_df1.rename(columns={"level_0": "ID"}, inplace=True)
metroGDPdatabysector_df1.set_index("ID", inplace=True)
metroGDPdatabysector_df1.fillna(value=0, inplace=True)
metroGDPdatabysector_df1.index = metroGDPdatabysector_df1.index.astype(int)
metroGDPdatabysector_df1.Percentage_Growth= metroGDPdatabysector_df1.Percentage_Growth.astype(int)
metroGDPdatabysector_df1.Dollar_Growth = metroGDPdatabysector_df1.Dollar_Growth.astype(int)
metroGDPdatabysector_df1


C:\Users\micha\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


KeyError: 'ID'

In [66]:
x1 = []
x2 = []
x3 = []


mean = 23 #total % gdp growth for the USA

for i in range(metroGDPdatabysector_df1.iloc[i:-1],11):
    i=0
    idx1,idx2,idx3=i,i+1, i+2                     
    row1,row2, row3=metroGDPdatabysector_df1.iloc[idx1],metroGDPdatabysector_df1.iloc[idx2],metroGDPdatabysector_df1.iloc[idx3]    
    x1.append(row1)                            
    x2.append(row2)                             
    x3.append(row3) 
    i += 3
print(x3)
# for row in metroGDPdatabysector_df1['Percentage Growth 05 to 10']:  
    
#     if row > mean:
      
#         betterthanaverage.append(row)
        
#     elif row < mean:
            
#             worsethanaverage.append(row)
            
#     else:
#             average.append(row)

# print(len(betterthanaverage))

TypeError: cannot do slice indexing on <class 'pandas.core.indexes.numeric.Int64Index'> with these indexers [23.05928944743914] of <class 'float'>

### Store JSON data into a DataFrame

In [ ]:
# json_file = "../Resources/customer_location.json"
# customer_location_df = pd.read_json(json_file)
# customer_location_df.head()

### Clean DataFrame

In [8]:
# new_customer_location_df = customer_location_df[["id", "address", "us_state"]].copy()
# new_customer_location_df.head()

### Connect to local database

In [9]:
connection_string = "root:password@localhost/econ_db"
engine = create_engine(f'mysql://{connection_string}')

### Check for tables

In [10]:
engine.table_names()

['metrogdp']

In [11]:
metroGDPdatabysector_df1.to_sql(name='metrogdp', con=engine, if_exists='append', index=True)

### Use pandas to load json converted DataFrame into database

In [272]:
new_customer_location_df.to_sql(name='customer_location', con=engine, if_exists='append', index=True, index_label='Geo')

NameError: name 'new_customer_location_df' is not defined

### Confirm data has been added by querying the customer_name table
* NOTE: can also check using pgAdmin

In [12]:
pd.read_sql_query('select * from metrogdp', con=engine).head()

,ID,index,GeoName,IndustryId,Description,2005,2006,2007,2008,2009,2010,Growth 05 to 10,Percentage Growth 05 to 10
0,0,78,United States (Metropolitan Portion),82.0,Government and government enterprises,1450899,1521538,1604465,1677648,1727701,1785466,334567.0,23.059290
1,1,85,United States (Metropolitan Portion),89.0,Private goods-producing industries,2294978,2456436,2553858,2531410,2286351,2376763,81785.0,3.563651
2,2,86,United States (Metropolitan Portion),90.0,Private services-providing industries,8044958,8507354,8896629,9020839,8942675,9256252,1211294.0,15.056561
3,3,165,"Abilene, TX (Metropolitan Statistical Area)",82.0,Government and government enterprises,1076,1118,1171,1161,1200,1266,190.0,17.657993
4,4,172,"Abilene, TX (Metropolitan Statistical Area)",89.0,Private goods-producing industries,1180,1333,1499,1832,0,1331,151.0,12.796610


### Confirm data has been added by querying the customer_location table

In [11]:
pd.read_sql_query('select * from customer_location', con=engine).head()

,id,address,us_state
0,1,043 Mockingbird Place,Indiana
1,2,4 Prentice Point,Indiana
2,3,46 Derek Junction,Texas
3,4,11966 Old Shore Place,Missouri
4,5,5 Evergreen Circle,New York
